In [5]:
from utils import load_data_geo

In [6]:
# Basic plots
%matplotlib inline
# Common imports and settings
import os
import sys

import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Markdown

# plt.rcParams['figure.figsize'] = [12, 8]

pd.set_option("display.max_rows", None)
# Datacube
import datacube
import xarray as xr
from datacube.utils import masking
from datacube.utils.cog import write_cog
from datacube.utils.rio import configure_s3_access
from deafrica_tools.classification import collect_training_data, predict_xr
from deafrica_tools.datahandling import mostcommon_crs, load_ard
from deafrica_tools.areaofinterest import define_area

# https://github.com/GeoscienceAustralia/dea-notebooks/tree/develop/Tools
from deafrica_tools.plotting import display_map, rgb, map_shapefile
from odc.io.cgroups import get_cpu_quota
import geopandas as gpd
from datacube.utils.geometry import CRS, Geometry
from deafrica_tools.plotting import rgb
from deafrica_tools.bandindices import calculate_indices

In [7]:
from datacube.utils.aws import configure_s3_access

configure_s3_access(aws_unsigned=False, requester_pays=True)

In [8]:
# region_path = "train/Soc Trang_Traning.shp"
region_path = "region/ST_region.shp"

In [9]:
region = load_data_geo(region_path)

In [10]:
region.head()

,ID_Reg,Regions,ID_Pro,Province,ID_Dis,District,ID_Com,Commune,merge,geometry
0,8.0,MKD,94,Soc Trang,941,Soc Trang City,31513,Ward 1,Soc Trang-Soc Trang City-Ward 1,"POLYGON Z ((105.87486 9.26280 0.00000, 105.873..."


In [11]:
region.explore(column="ID_Pro", legend=False)
# region.explore(column="Name", legend=False)

In [12]:
train_path = "train/Soc Trang_Traning.shp"
train = load_data_geo(train_path)
train.head()

,Name,geometry
0,Water1,POINT (620223.628 1072681.101)
1,Water1,POINT (621057.521 1071074.602)
2,Water1,POINT (621891.602 1069430.621)
3,Water1,POINT (622762.564 1068048.689)
4,Water1,POINT (623256.484 1066777.765)


In [13]:
train.explore(column="Name", legend=True)

In [14]:
dc = datacube.Datacube(app='crop_health_evi')

In [15]:
geopolygon = gpd.read_file(region_path)

In [16]:
gdf = gpd.read_file(region_path)
lat_range = (gdf.total_bounds[1], gdf.total_bounds[3])
lon_range = (gdf.total_bounds[0], gdf.total_bounds[2])

In [17]:
display_map(x=lon_range, y=lat_range)

In [18]:
query = {
    'x': lon_range,
    'y': lat_range,
    'time': ("2022-01-01", "2022-12-31"),
    'resolution': (-10, 10),
    # "output_crs": output_crs,
}
query

{'x': (105.54366239999999, 106.33510295999999),
 'y': (9.243697679999999, 9.93806676),
 'time': ('2022-01-01', '2022-12-31'),
 'resolution': (-10, 10)}

In [19]:
# ds1 = load_ard(dc=dc,
#               products=['s2_l2a'],
#               min_gooddata=0.99,
#               measurements=['red', 'green', 'blue', 'nir'],
#               output_crs=output_crs, 
#               **query)

In [ ]:
import time
t_time = time.time()
query = {
    'x': lon_range,
    'y': lat_range,
    'time': ("2022-01-01", "2022-12-31"),
    "measurements": ['red', 'green', 'blue', 'nir'],
    'resolution': (-10, 10),
}
output_crs = mostcommon_crs(dc=dc, product='s2_l2a', query=query)
query["output_crs"] = output_crs
ds = dc.load(product="s2_l2a", group_by='solar_day', **query)
print(time.time() - t_time)

In [ ]:
display(ds)

In [ ]:
rgb(ds, col="time")

In [ ]:
ds = calculate_indices(ds, index='NDVI', satellite_mission='s2')

In [ ]:
ds.NDVI.plot(col='time',
           cmap='RdYlGn',
           size=6,
col_wrap=2)

In [ ]:
display(ds)